In [54]:
import pandas as pd
import numpy as np

In [55]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

# 特征处理

In [56]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['day'] = all_data['date'].dt.day

In [57]:

features = ['year','month','day','hour', 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP']
labels = ['Label1', 'Label2']

In [58]:
train = all_data[~all_data['is_test']].copy(deep=True)  # .fillna(-1)
test = all_data[all_data['is_test']].copy(deep=True)  # .fillna(-1)

In [59]:
np.sum(all_data[labels[0]].isna())

115412

In [64]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
for label in labels:
    train[label] = train[label].fillna(0)
    tmp = train[~train[label].isna()].reset_index(drop=True).copy(deep=True)
    print(len(tmp))
    X_train, X_test, y_train, y_test = train_test_split(tmp[features], tmp[label], train_size=0.2, random_state=1)
    model = lgb.LGBMRegressor(
        boosting="gbdt",
        max_depth=6,
        learning_rate=0.001,
        n_estimators=20000,
        min_child_weight=1,
        min_data_in_leaf=40,
        subsample = 0.7,
        feature_fraction=0.7,
        bagging_seed=1,
        reg_alpha=1,
        reg_lambda=1,  # 此处不改了
        min_sum_hessian_in_leaf=0.01,
        random_state=1212
    )

    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['mae'],
          early_stopping_rounds=150, verbose=200)

    test[label] = model.predict(test[features])

140480
Training until validation scores don't improve for 150 rounds
[200]	training's l1: 4370.8	training's l2: 3.05616e+07	valid_1's l1: 4374.76	valid_1's l2: 3.05835e+07
[400]	training's l1: 3723.32	training's l2: 2.31919e+07	valid_1's l1: 3729.56	valid_1's l2: 2.32611e+07
[600]	training's l1: 3181.54	training's l2: 1.79252e+07	valid_1's l1: 3189.93	valid_1's l2: 1.80253e+07
[800]	training's l1: 2730.98	training's l2: 1.41426e+07	valid_1's l1: 2741.01	valid_1's l2: 1.42601e+07
[1000]	training's l1: 2353.94	training's l2: 1.13745e+07	valid_1's l1: 2365.25	valid_1's l2: 1.14968e+07
[1200]	training's l1: 2037.44	training's l2: 9.33413e+06	valid_1's l1: 2050.36	valid_1's l2: 9.46165e+06
[1400]	training's l1: 1770.66	training's l2: 7.78991e+06	valid_1's l1: 1785.29	valid_1's l2: 7.92713e+06
[1600]	training's l1: 1549.51	training's l2: 6.63062e+06	valid_1's l1: 1566.06	valid_1's l2: 6.77063e+06
[1800]	training's l1: 1372.02	training's l2: 5.77106e+06	valid_1's l1: 1390.32	valid_1's l2: 5.9

In [61]:
test[['time'] + labels].to_csv("./res/base_line.csv", index=False)

In [65]:
loss = np.sqrt(1.26355e+06) + np.sqrt(945172)/2
1000/(1+loss)

0.6206642014686299